In [ ]:
import cv2
import os
import numpy as np

def create_noisy_images(original_folder, noisy_folder, noise_level=25):
    if not os.path.exists(noisy_folder):
        os.makedirs(noisy_folder)

    for img_name in os.listdir(original_folder):
        img_path = os.path.join(original_folder, img_name)
        img = cv2.imread(img_path)

        # Tạo tiếng ồn Gaussian
        noise = np.random.randn(*img.shape) * noise_level

        # Thêm tiếng ồn vào ảnh gốc
        noisy_img = np.clip(img + noise, 0, 255).astype(np.uint8)

        # Lưu ảnh có tiếng ồn
        cv2.imwrite(os.path.join(noisy_folder, img_name), noisy_img)

# Đường dẫn đến thư mục chứa ảnh gốc
original_folder = './NoisingImages/original'
noisy_folder = './NoisingImages/noisy'

create_noisy_images(original_folder, noisy_folder)


: 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.models import Model
import os

# Class để đọc ảnh và trả về batch
class ImageDataGenerator(Sequence):
    def __init__(self, noisy_folder_path, clean_folder_path, batch_size=16, img_size=(256, 256)):
        self.noisy_folder_path = noisy_folder_path
        self.clean_folder_path = clean_folder_path
        self.batch_size = batch_size
        self.img_size = img_size
        self.noisy_images = os.listdir(noisy_folder_path)

    def __len__(self):
        return int(np.ceil(len(self.noisy_images) / self.batch_size))

    def __getitem__(self, index):
        batch_noisy = self.noisy_images[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = []
        Y_batch = []

        for img_name in batch_noisy:
            noisy_path = os.path.join(self.noisy_folder_path, img_name)
            clean_path = os.path.join(self.clean_folder_path, img_name)

            # Load và xử lý ảnh có tiếng ồn
            img_noisy = load_img(noisy_path, target_size=self.img_size)
            img_noisy = img_to_array(img_noisy) / 255.0
            X_batch.append(img_noisy)

            # Load và xử lý ảnh gốc
            img_clean = load_img(clean_path, target_size=self.img_size)
            img_clean = img_to_array(img_clean) / 255.0
            Y_batch.append(img_clean)

        return np.array(X_batch, dtype=np.float32), np.array(Y_batch, dtype=np.float32)

# Hàm xây dựng mô hình CNN đơn giản cho Denoising
def build_model():
    input_img = Input(shape=(256, 256, 3))
    
    # Các lớp Conv2D để khôi phục ảnh
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)  # Đầu ra

    # Định nghĩa mô hình
    model = Model(inputs=input_img, outputs=x)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

# Đường dẫn đến thư mục chứa ảnh có tiếng ồn và ảnh sạch
noisy_folder_path = './DenoisingImages/noisy'
clean_folder_path = './DenoisingImages/clean'

# Khởi tạo ImageDataGenerator để đọc dữ liệu
train_data_gen = ImageDataGenerator(noisy_folder_path, clean_folder_path, batch_size=16)

# Xây dựng mô hình
model = build_model()

# Cấu hình EarlyStopping để dừng huấn luyện nếu không có cải thiện
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Huấn luyện mô hình
model.fit(train_data_gen, epochs=50, callbacks=[early_stopping])

# Lưu mô hình sau khi huấn luyện
model.save('image_denoising_model.h5')
